# **Transacciones MIMIC IV**

In [ ]:
%pip install mlxtend

## Cargar el fichero .parquet de datos 

Dada la magnitud de los datos se hace uso de un dataframe de Spark para convertirlos en un formato adecuado para el análisis de reglas de asociación

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, regexp_extract, count

# Detener sesión anterior
try:
    spark.stop()
except:
    pass


spark = SparkSession.builder \
    .appName("Preparación dataset para entrenamiento del modelo predictivo") \
    .config("spark.master", "local[*]") \
    .config("spark.driver.memory", "20g") \
    .config("spark.executor.memory", "20g") \
    .getOrCreate()

In [ ]:
base_path = f"./data"
input_path = f"{base_path}/resultados/dataset_final.parquet"

In [ ]:
df_dataset = spark.read.parquet(input_path)

In [ ]:
df_dataset.show(10)

## Extraer datos para el algoritmo de Reglas de Asociación

### Consultar con Spark SQL

In [ ]:
df_dataset.createOrReplaceTempView("resultados")

In [ ]:
df_association_rules = df_dataset.select('*')
df_association_rules.show()

In [ ]:
# Admisiones unicas y que corresponderán a transacciones
# Contar las admisiones únicas
unique_admissions = spark.sql("SELECT COUNT(DISTINCT id_ingreso) AS total_admisiones FROM resultados")

unique_admissions.show()

In [ ]:
df_ages = spark.sql("select id_ingreso, edad from resultados group by id_ingreso, edad order by edad").toPandas()

In [ ]:
df_ages.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Asegurar que la columna de edad es numérica
df_ages["edad"] = df_ages["edad"].astype(int)

# Definir el rango de edades de 20 a 90 con intervalos de 10
edad_min = 20
edad_max = 90
bins = np.arange(edad_min, edad_max + 10, 10)

plt.figure(figsize=(10, 6))

# Generar histograma
df_ages["edad"].hist(
    bins=bins, color='#99CC99', alpha=0.7, edgecolor='white', density=True
)

# Ajustar etiquetas del eje X
plt.xticks(
    bins[:-1] + 5,
    labels=[f"{int(b)}-{int(b+9)}" for b in bins[:-1]],
    rotation=0
)

plt.xlabel("Edad (Rangos)")
plt.ylabel("Frecuencia")
plt.title("Distribución de Edades")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.xlim(edad_min, edad_max) 

plt.show()

In [ ]:
from pyspark.sql.functions import when

df_association_rules = df_association_rules.withColumn(
    "edad_categoria",
    when((df_association_rules["edad"] >= 18) & (df_association_rules["edad"] <= 39), "Joven")
    .when((df_association_rules["edad"] >= 40) & (df_association_rules["edad"] <= 64), "Adulto joven")
    .when((df_association_rules["edad"] >= 65) & (df_association_rules["edad"] <= 79), "Adulto mayor")
    .otherwise("Anciano")  # Para 80+ años
)

df_association_rules.select("edad", "edad_categoria").show(10)

In [ ]:
df_association_rules = df_association_rules.drop("edad")
display(df_association_rules)

In [ ]:
df_association_rules.createOrReplaceTempView("resultados")

### Tratar valores nulos

In [ ]:
from pyspark.sql import functions as F

# Contar el número de valores nulos por columna
null_counts = df_association_rules.select(
    [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_association_rules.columns]
)

# Mostrar el resultado
display(null_counts)

In [ ]:
df_association_rules = df_association_rules.fillna({
    "estado_civil": "estado_civil_desconocido",
    "tipo_seguro": "tipo_seguro_desconocido"
})

In [ ]:
df_association_rules.createOrReplaceTempView("resultados")

## Transformar a la estructura de datos adecuada para el algoritmo de minería de reglas

In [ ]:
from pyspark.sql.functions import collect_set, concat_ws

# Aplicar la lógica al dataframe en Spark
df_items = df_association_rules.groupBy("id_ingreso").agg(
    concat_ws(",", 
        collect_set("id_prueba"), 
        collect_set("dominio_icd"),
        collect_set("estado_civil"),
        collect_set("tipo_seguro"),
        collect_set("grupo_poblacional"),
        collect_set("muerte_durante_ingreso"),
        collect_set("sexo"),
        collect_set("edad_categoria")
    ).alias("items")
)

# Mostrar el resultado
df_items.show(truncate=False)

El resultado es un **dataset en formato transaccional**, donde cada transacción (en este caso, un ingreso hospitalario identificado como id_ingreso) esté asociada a un conjunto de ítems (pruebas bioquímicas, diagnósticos, características sociodemográficas)

In [ ]:
df_items.write.mode("overwrite").parquet(f"{base_path}/resultados/dataset_reglas.parquet")

In [ ]:
csv_path = f"{base_path}/resultados/dataset_reglas.csv"

In [ ]:
df_items.coalesce(1).write.mode("overwrite").option("header", "true").csv(csv_path)

In [ ]:
# Cargar el archivo CSV
df_transactions = spark.read.csv(
    csv_path,
    header=True,
    inferSchema=True
)

In [ ]:
from pyspark.sql import functions as F

# Comprobar si hay valores nulos en las columnas
df_transactions.select(
    [
        F.sum(F.col("id_ingreso").isNull().cast("int")).alias("null_id_ingreso"),
        F.sum(F.col("items").isNull().cast("int")).alias("null_items"),
    ]
).show()

In [ ]:
# Mostrar el esquema para verificar que la columna `items` es un array o lista
df_transactions.printSchema()

Se necesita transformar la cadena en una lista para que FP-Growth pueda procesarla

In [ ]:
from pyspark.sql.functions import split, col

# Convertir la columna 'items' de STRING a ARRAY<STRING>
df_transactions = df_transactions.withColumn("items", split(col("items"), ","))

# Verificar el esquema y los datos
df_transactions.printSchema()
df_transactions.show()

In [ ]:
from pyspark.ml.fpm import FPGrowth

# Crear el modelo FP-Growth y entrenarlo
fp_growth = FPGrowth(itemsCol="items", minSupport=0.05, minConfidence=0.7)

model = fp_growth.fit(df_transactions)

In [ ]:
# Conjuntos frecuentes
frequent_itemsets = model.freqItemsets

In [ ]:
# Reglas de asociación
association_rules = model.associationRules.persist()

In [ ]:
# Contar número de reglas de asociación
association_rules.take(1)  
print(association_rules.count())

In [ ]:
# Guardar DF con reglas de asociacion
rules_path = f"{base_path}/resultados/fpgrowth_rules.parquet"

association_rules.write.mode("overwrite").parquet(rules_path)

In [ ]:
# Leer el dataset guardado
saved_rules = spark.read.parquet(rules_path)

# Mostrar las primeras filas
saved_rules.show(5)

In [ ]:
import gbc